In [88]:
from keras.layers import Input,Dense,Embedding,Lambda,TimeDistributed,LSTM, Reshape, Dropout
from keras.models import Model
from TransformerEncoder import TransformerEncoder
from TrainablePositionalEmbeddings import TransformerPositionalEmbedding
import os
import pickle
from sklearn.model_selection import train_test_split
from keras.optimizers import Nadam, Adam
from Attention import MultiHeadedAttention
from sklearn.preprocessing import StandardScaler, MinMaxScaler

## data processing

In [6]:
data_df = pd.read_csv('data/sku_week_20191024_v2.csv')

In [7]:
data_df

,sku_id,sku_name,cio_id,cio_name,brand,sub_brand,time,y
0,25010,哈尔滨冰纯9.1度600ML1X12纸箱回瓶装,100005.0,"HARBIN ICE哈尔滨冰纯BBT大瓶600ml , (12)",HICE,哈尔滨冰纯,2015-02,1423.872
1,25010,哈尔滨冰纯9.1度600ML1X12纸箱回瓶装,100005.0,"HARBIN ICE哈尔滨冰纯BBT大瓶600ml , (12)",HICE,哈尔滨冰纯,2015-03,2239.488
2,25010,哈尔滨冰纯9.1度600ML1X12纸箱回瓶装,100005.0,"HARBIN ICE哈尔滨冰纯BBT大瓶600ml , (12)",HICE,哈尔滨冰纯,2015-04,1824.768
3,25010,哈尔滨冰纯9.1度600ML1X12纸箱回瓶装,100005.0,"HARBIN ICE哈尔滨冰纯BBT大瓶600ml , (12)",HICE,哈尔滨冰纯,2015-05,1963.008
4,25010,哈尔滨冰纯9.1度600ML1X12纸箱回瓶装,100005.0,"HARBIN ICE哈尔滨冰纯BBT大瓶600ml , (12)",HICE,哈尔滨冰纯,2015-06,1817.856
...,...,...,...,...,...,...,...,...
42041,74715,福佳玫瑰红啤酒10.0度248ML1x24纸箱OW瓶装,106035.0,"HOEGAARDEN 福佳红玫SBT小瓶248ml , (24)",OTHE,福佳红玫,2019-35,20.713
42042,74715,福佳玫瑰红啤酒10.0度248ML1x24纸箱OW瓶装,106035.0,"HOEGAARDEN 福佳红玫SBT小瓶248ml , (24)",OTHE,福佳红玫,2019-36,0.000
42043,74715,福佳玫瑰红啤酒10.0度248ML1x24纸箱OW瓶装,106035.0,"HOEGAARDEN 福佳红玫SBT小瓶248ml , (24)",OTHE,福佳红玫,2019-37,11.905
42044,74715,福佳玫瑰红啤酒10.0度248ML1x24纸箱OW瓶装,106035.0,"HOEGAARDEN 福佳红玫SBT小瓶248ml , (24)",OTHE,福佳红玫,2019-38,44.461


In [8]:
c1_list = [58465, 25249, 36520]
a3_list = [30851, 59546, 32194, 31621, 48341]

In [9]:
key_df = data_df[data_df['sku_id'].isin(c1_list) | data_df['sku_id'].isin(a3_list)]

In [11]:
key_df['y'].median()

434.3045000000001

24211.944

## Model

In [270]:
input_seq_len = 16
input_dim = 1
hidden_dim = 100
positional_ff_dim = 256
dropout_rate = 0.1
output_seq_len = 1
output_dim = 1

output_dir = './output'

In [271]:
def time_series_model_LSTM_Transformer():
    
    encoder_inputs = Input((input_seq_len,input_dim,),dtype='float32',name='Encoder_input')
#     encoder = [LSTM(hidden_dim,return_state=True,return_sequences=True,name='Encoder'+str(i)) for i in range(num_layers)]
    
    positional_embedding_layer    = TransformerPositionalEmbedding(name='Positional_embedding')
    next_step_input               = positional_embedding_layer(encoder_inputs)
    
    next_step_input,attention =TransformerEncoder(hidden_dim,
                                                             2,
                                                             hidden_dim,
                                                             hidden_dim,
                                                             positional_ff_dim,
                                                             dropout_rate= 0.1,
                                                             name= 'Transformer')(next_step_input)
    
    next_step_input,attention =TransformerEncoder(hidden_dim,
                                                         2,
                                                         hidden_dim,
                                                         hidden_dim,
                                                         positional_ff_dim,
                                                         dropout_rate= 0.1,
                                                         name= 'Transformer1')(next_step_input)
    '''
    
    next_step_input,attention =TransformerEncoder(hidden_dim,
                                                         2,
                                                         hidden_dim,
                                                         hidden_dim,
                                                         positional_ff_dim,
                                                         dropout_rate= 0.1,
                                                         name= 'Transformer2')(next_step_input)
    '''
    
    
    
    
    state_h = Lambda(lambda x : x[:,0, :] )(next_step_input)
    state_c = Lambda(lambda x : x[:,1, :] )(next_step_input)
    
    #print(state_h)
    decoder_inputs = Input(tensor=encoder_inputs[:,-(output_seq_len):,:])
    decoder = LSTM(hidden_dim,return_sequences=True,return_state=True,name='Decoder')
    predicted_values,_,_ = decoder(decoder_inputs,initial_state = [state_h,state_c])
#     print(predicted_values)
#     predicted_values = Lambda(lambda x : x[:,:output_seq_len, :] )(next_step_input)
    reshaped_outputs = TimeDistributed(Dense(output_dim))(predicted_values)
    #next_step_input = Lambda(lambda x: x[:,0])(next_step_input)
    #reshaped_outputs = Dense(output_dim)(next_step_input)
    
    #predicted_values, _, _ = decoder(next_step_input)
    #print(predicted_values)
    #predicted_values = Lambda(lambda x: x[:,0])(predicted_values)
    #reshaped_outputs = Dense(output_dim)(predicted_values)
    
    return Model(inputs= [encoder_inputs,decoder_inputs],output=[reshaped_outputs])
    #return Model(inputs=[encoder_inputs], output = [reshaped_outputs])

In [171]:
def LSTM_time_series_model():
    inputs = Input((input_seq_len,input_dim,))
    x = LSTM(hidden_dim, return_sequences=True)(inputs)
    attention_layer = MultiHeadedAttention(hidden_dim,1,hidden_dim,hidden_dim,0.1, name='ATT')
    x, _ = attention_layer(x)
    x = Dropout(0.1)(x)
    #x = Lambda(lambda x: x[:,0])(x)
    x = Dense(output_dim)(x)
    x = Reshape((input_seq_len,))(x)
    outputs = Dense(output_dim)(x)
    
    
    return Model(inputs=[inputs], output = [outputs])
    
    
    

In [99]:
model = LSTM_time_series_model()

/home/adonis/.keras/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  del sys.path[0]


In [272]:
model = time_series_model_LSTM_Transformer()

/home/adonis/.keras/lib/python3.7/site-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [226]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_input (InputLayer)      (None, 16, 1)        0                                            
__________________________________________________________________________________________________
Positional_embedding (Transform (None, 16, 1)        16          Encoder_input[0][0]              
__________________________________________________________________________________________________
Transformer_QueryLayer (TimeDis (None, 16, 200)      200         Positional_embedding[0][0]       
__________________________________________________________________________________________________
Transformer_KeyLayer (TimeDistr (None, 16, 200)      200         Positional_embedding[0][0]       
___________________________________________________________________________________________

In [273]:
model.compile(optimizer=Nadam(), loss='mean_squared_error')

In [228]:
with open('data/SKU_30851_data.pkl','rb') as f:
    data = pickle.load(f)
scaler = MinMaxScaler()
data = scaler.fit_transform(data)
data_train, data_test = train_test_split(data, test_size=0.05, random_state=42)
x_train = data_train[:,:-1]
y_train = data_train[:,-1]
x_test = data_test[:,:-1]
y_test = data_test[:,-1]

x_train = x_train.reshape(y_train.shape[0],16,1)
x_test = x_test.reshape(y_test.shape[0],16,1)

y_train = y_train.reshape(y_train.shape[0],1,1)
y_test = y_test.reshape(y_test.shape[0],1,1)


In [274]:
history = model.fit(x_train,y_train, epochs=300)

Epoch 1/300
203/203 [==============================] - 1s 6ms/step - loss: 0.8635
Epoch 2/300
203/203 [==============================] - 0s 444us/step - loss: 0.0666
Epoch 3/300
203/203 [==============================] - 0s 365us/step - loss: 0.0493
Epoch 4/300
203/203 [==============================] - 0s 359us/step - loss: 0.0522
Epoch 5/300
203/203 [==============================] - 0s 358us/step - loss: 0.0709
Epoch 6/300
203/203 [==============================] - 0s 357us/step - loss: 0.0556
Epoch 7/300
203/203 [==============================] - 0s 360us/step - loss: 0.0448
Epoch 8/300
203/203 [==============================] - 0s 360us/step - loss: 0.0556
Epoch 9/300
203/203 [==============================] - 0s 354us/step - loss: 0.0518
Epoch 10/300
203/203 [==============================] - 0s 361us/step - loss: 0.0439
Epoch 11/300
203/203 [==============================] - 0s 355us/step - loss: 0.0444
Epoch 12/300
203/203 [==============================] - 0s 358us/step - loss

203/203 [==============================] - 0s 357us/step - loss: 0.0381
Epoch 98/300
203/203 [==============================] - 0s 363us/step - loss: 0.0398
Epoch 99/300
203/203 [==============================] - 0s 363us/step - loss: 0.0387
Epoch 100/300
203/203 [==============================] - 0s 356us/step - loss: 0.0391
Epoch 101/300
203/203 [==============================] - 0s 364us/step - loss: 0.0401
Epoch 102/300
203/203 [==============================] - 0s 358us/step - loss: 0.0395
Epoch 103/300
203/203 [==============================] - 0s 371us/step - loss: 0.0401
Epoch 104/300
203/203 [==============================] - 0s 362us/step - loss: 0.0386
Epoch 105/300
203/203 [==============================] - 0s 374us/step - loss: 0.0381
Epoch 106/300
203/203 [==============================] - 0s 355us/step - loss: 0.0372
Epoch 107/300
203/203 [==============================] - 0s 360us/step - loss: 0.0392
Epoch 108/300
203/203 [==============================] - 0s 356us/step

203/203 [==============================] - 0s 360us/step - loss: 0.0153
Epoch 193/300
203/203 [==============================] - 0s 356us/step - loss: 0.0150
Epoch 194/300
203/203 [==============================] - 0s 502us/step - loss: 0.0171
Epoch 195/300
203/203 [==============================] - 0s 359us/step - loss: 0.0149
Epoch 196/300
203/203 [==============================] - 0s 357us/step - loss: 0.0160
Epoch 197/300
203/203 [==============================] - 0s 361us/step - loss: 0.0208
Epoch 198/300
203/203 [==============================] - 0s 360us/step - loss: 0.0212
Epoch 199/300
203/203 [==============================] - 0s 360us/step - loss: 0.0219
Epoch 200/300
203/203 [==============================] - 0s 356us/step - loss: 0.0201
Epoch 201/300
203/203 [==============================] - 0s 356us/step - loss: 0.0198
Epoch 202/300
203/203 [==============================] - 0s 365us/step - loss: 0.0186
Epoch 203/300
203/203 [==============================] - 0s 371us/st

203/203 [==============================] - 0s 359us/step - loss: 0.0100
Epoch 288/300
203/203 [==============================] - 0s 378us/step - loss: 0.0107
Epoch 289/300
203/203 [==============================] - 0s 358us/step - loss: 0.0113
Epoch 290/300
203/203 [==============================] - 0s 356us/step - loss: 0.0109
Epoch 291/300
203/203 [==============================] - 0s 363us/step - loss: 0.0108
Epoch 292/300
203/203 [==============================] - 0s 358us/step - loss: 0.0106
Epoch 293/300
203/203 [==============================] - 0s 358us/step - loss: 0.0109
Epoch 294/300
203/203 [==============================] - 0s 356us/step - loss: 0.0106
Epoch 295/300
203/203 [==============================] - 0s 358us/step - loss: 0.0108
Epoch 296/300
203/203 [==============================] - 0s 358us/step - loss: 0.0120
Epoch 297/300
203/203 [==============================] - 0s 354us/step - loss: 0.0117
Epoch 298/300
203/203 [==============================] - 0s 388us/st

In [275]:
model.evaluate(x_test, y_test)

11/11 [==============================] - 0s 14ms/step


0.028487909585237503

In [276]:
pred = model.predict(x_test)

In [277]:
pred = pred.reshape(y_test.shape[0],)

In [278]:
pred

array([0.18846312, 0.12933911, 0.13643132, 0.1417519 , 0.148172  ,
       0.08519126, 0.08631994, 0.04738076, 0.03923615, 0.5686521 ,
       0.07338565], dtype=float32)

In [218]:
pred = np.concatenate((pred, np.array([0.0]*6)), axis=0)

In [222]:
y_test = np.concatenate((y_test.reshape(y_test.shape[0],), np.array([0.0]*6)), axis=0)

In [205]:
y_test.reshape(y_test.shape[0],)

array([0.41632653, 0.00734694, 0.29183673, 0.22040816, 0.49102041,
       0.09959184, 0.06      , 0.21428571, 0.01387755, 0.83061224,
       0.0477551 ])

In [220]:
scaler.inverse_transform(pred.reshape(1,-1))

array([[3170.05202937, 2516.40255368, 2734.9455083 , 1980.66542333,
        3499.70189613,  897.50326225, 2756.4945091 , 2925.22723782,
        2742.4194907 , 8159.13222241, 2267.52483058,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ]])

In [223]:
scaler.inverse_transform(y_test.reshape(1,-1))

array([[ 6609.6 ,   116.64,  4633.2 ,  3499.2 ,  7795.44,  1581.12,
          952.56,  3402.  ,   220.32, 13186.8 ,   758.16,     0.  ,
            0.  ,     0.  ,     0.  ,     0.  ,     0.  ]])

In [180]:
import numpy as np

In [279]:
acc = 1 - np.mean(abs(pred - y_test.reshape(y_test.shape[0],)) / (y_test.reshape(y_test.shape[0],)+0.01))

In [280]:
acc

-0.10695593344954979

In [192]:
abs(pred - y_test.reshape(24,))# / (y_test.reshape(24,))

array([0.89205315, 0.87075845, 0.24585865, 0.03594618, 1.60796462,
       0.14524652, 0.11866554, 0.30949681, 1.99716268, 0.48063638,
       0.93649645, 0.22187629, 1.64933452, 1.38795963, 0.44200471,
       1.02982953, 1.07476275, 0.6195276 , 1.02828431, 0.23644275,
       0.98890043, 0.95346081, 0.96893199, 0.37468421])